# 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
- 케라스의 임베딩 층(embedding layer)와 사전 훈련된 워드 임베딩(pre-trained word embedding)을 가져와 사용하는 것을 비교할 것이다

- 자연어 처리를 하려고 할때, 갖고 있는 훈련 데이터의 단어들을 임베딩 층을 구현하여 임베딩 벡터로 학습한다.
    - Keras 에서는 Embedding() 도구를 이용하여 구현한다.
- 위키피디아 등과 같이 방대한 코퍼스를 가지고 Word2Vec, FastText, GloVe를 통해 미리 훈련된 임베딩 벡터를 불러오는 방법도 있다.


### 1. 케라스 임베딩 층 (Keras Embedding Layer)
- Embedding()

#### 1-1) 임베딩 층은 룩업 테이블이다
- 임베딩 층의 입력으로 사용하기 위해서는, 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야 한다.
    - 어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집 벡터
    - 임베딩 층 : 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑하고 이는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련된다. ( 가중치 업데이트되는 과정)

<p align='center'><img src='https://wikidocs.net/images/page/33793/lookup_table.PNG'></p>

- 위 그림은 단어'great'를 정수 인코딩하여 해당 임베딩 벡터를 꺼내온다
    - 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 great의 임베딩 벡터값이 학습된다.
    - 단어를 정수 인코딩까지만 진행하여 임베딩 층의 입력으로 사용하여 룩업 테이블 결과인, 임베딩 벡터를 리턴해준다

```python
# 임베딩 층 구현 코드
vocab_size = 20000
output_dim = 128
input_length = 500

v=Embedding(vocab_size, output_dim, input_length = input_length)
```

- vocab_size : 텍스트 데이터의 전체 단어 집합의 크기
- output_dim : 워드 임베딩 후의 임베딩 벡터의 차원
- input_length : 입력 시퀀스의 길이, (각 샘플의 길이가 500이라면 input_length 는 500)

- Embedding 으로는 (number of samples, input_length)로 2D 텐서를 input으로 입력 받고
- output으로 (number of sammples, input_length, embedding word dimentionality) 3D텐서를 리턴한다.

In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 긍정(1) 부정(0)
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 
             'supreme quality', 'bad', 'highly respectable']
y_train = [1,0,0,1,1,0,1]

### 훈련 데이터 전처리

# 1. 케라스의 토크나이저를 사용하여 단어 집합을 만들고 그 크기를 확인한다.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) +1 # 패딩을 고려하여 +1 해준다
print("단어 집합 크기 : ", vocab_size)

# 2. 각 문장에 대해서 정수 인코딩 수행
X_encoded = tokenizer.texts_to_sequences(sentences)
print("정수 인코딩 결과 : ", X_encoded)

# 3. 가장 길이가 긴 문장의 길이 구하기
max_len = max(len(l) for l in X_encoded)
print("최대 길이 : ", max_len)

# 4. 최대 길이로 모든 샘플에 대해서 패딩 진행하기
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print("패딩 결과 :")
print(X_train)




단어 집합 크기 :  16
정수 인코딩 결과 :  [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]
최대 길이 :  4
패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [7]:
### 이진 분류 모델 설계

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 205ms/step - loss: 0.6974 - acc: 0.4286
Epoch 2/100
1/1 [==============================] - 0s 15ms/step - loss: 0.6962 - acc: 0.4286
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6949 - acc: 0.4286
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6937 - acc: 0.4286
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6924 - acc: 0.5714
Epoch 6/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6912 - acc: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6900 - acc: 0.5714
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6887 - acc: 0.5714


2022-02-16 23:50:49.515583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6875 - acc: 0.5714
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6863 - acc: 0.5714
Epoch 11/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6850 - acc: 0.5714
Epoch 12/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6838 - acc: 0.5714
Epoch 13/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6826 - acc: 0.7143
Epoch 14/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6814 - acc: 0.7143
Epoch 15/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6801 - acc: 0.7143
Epoch 16/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6789 - acc: 0.7143
Epoch 17/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6777 - acc: 0.7143
Epoch 18/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6764 - acc: 0.7143
Epoch 19/100
1/1 [==============================] - 0s 10ms/s

### 2. 사전 훈련된 워드 임베딩 사용하기
- Keras의 Embedding()을 임베딩 벡터값을 학습하기도 하지만, 훈련된 워드 임베딩을 가져와서 사용하는 경우도 있다.
- GloVe 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip
- Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

#### 2-1) 사전 훈련된 GloVe 사용하기
- glove.6B.100d.txt 파일 사용

In [8]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

embedding_dict = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]

    # 100개의 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

400000개의 Embedding vector가 있습니다.
[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0

In [9]:
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 100)


In [10]:
print(tokenizer.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [11]:
print("단어 great의 맵핑된 정수 :", tokenizer.word_index['great'])

단어 great의 맵핑된 정수 : 2


In [12]:
# 사전 훈련된 GloVe의 벡터값 확인
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

- 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑한 후 'great'의 벡터값이 의도한 인덱스의 위치에 삽입되었는지 확인해보겠습니다.

In [13]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [16]:
# 아까 great 는 인덱스가 2였기 때문에 아래와 동일하다
if all(embedding_matrix[2] == embedding_dict['great']):
    print("일치")

일치


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.6330 - acc: 0.7143 - 174ms/epoch - 174ms/step
Epoch 2/100
1/1 - 0s - loss: 0.6137 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 3/100
1/1 - 0s - loss: 0.5951 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 4/100
1/1 - 0s - loss: 0.5774 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 5/100
1/1 - 0s - loss: 0.5604 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 6/100
1/1 - 0s - loss: 0.5441 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 7/100
1/1 - 0s - loss: 0.5285 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 8/100
1/1 - 0s - loss: 0.5136 - acc: 0.8571 - 6ms/epoch - 6ms/step
Epoch 9/100
1/1 - 0s - loss: 0.4994 - acc: 0.8571 - 6ms/epoch - 6ms/step
Epoch 10/100
1/1 - 0s - loss: 0.4857 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 11/100
1/1 - 0s - loss: 0.4726 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 12/100
1/1 - 0s - loss: 0.4601 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 13/100
1/1 - 0s - loss: 0.4481 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 14/100
1/1 - 0s - loss: 0.4365 - acc:

2022-02-17 00:37:11.487519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 16/100
1/1 - 0s - loss: 0.4148 - acc: 1.0000 - 7ms/epoch - 7ms/step
Epoch 17/100
1/1 - 0s - loss: 0.4045 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 18/100
1/1 - 0s - loss: 0.3946 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 19/100
1/1 - 0s - loss: 0.3851 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 20/100
1/1 - 0s - loss: 0.3758 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 21/100
1/1 - 0s - loss: 0.3669 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 22/100
1/1 - 0s - loss: 0.3583 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 23/100
1/1 - 0s - loss: 0.3499 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 24/100
1/1 - 0s - loss: 0.3418 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 25/100
1/1 - 0s - loss: 0.3340 - acc: 1.0000 - 5ms/epoch - 5ms/step
Epoch 26/100
1/1 - 0s - loss: 0.3264 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 27/100
1/1 - 0s - loss: 0.3190 - acc: 1.0000 - 6ms/epoch - 6ms/step
Epoch 28/100
1/1 - 0s - loss: 0.3118 - acc: 1.0000 - 10ms/epoch - 10ms/step
Epoch 29/100
1/1 - 0s - loss: 0.3049